In [36]:
import production as p
import os
import inspect
import sys
import pandas as pd

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step1: Read the LLM-ready data

In [28]:
df_user_data,neutral_sentences, user_sentences = p.read_input_data()

In [29]:
neutral_sentences

,idSentence,sentences
0,0,I’m all about that food. I usually kick off th...
1,1,Just getting my vitamins in at the school cant...
2,2,De Pizzabakkers sell this vegan pizza with che...
3,3,I entered the world of vegan foods lately. Nex...
4,4,This vegan fried chicken from KFC is on the sp...
5,5,Just having this vegan hotdog from the school ...
6,6,"Just had my first vegan cake at ""groene bakker..."
7,7,This vegan chocolate is on point. Its with oat...
8,8,I'm all about fast food. I'm having some fries...
9,9,"I went to the ""Groene Burger"" fast food restau..."


In [30]:
user_sentences

,user,original,your_text,idSentence
0,U0,I’m all about that food. I usually kick off th...,I love food! I really like to have beans with ...,0
1,U0,Just getting my vitamins in at the school cant...,"I am going for a something healthy today, quin...",1
2,U0,De Pizzabakkers sell this vegan pizza with che...,Have you tried the new vegan mozzarella pizza ...,2
3,U0,I entered the world of vegan foods lately. Nex...,I have so many vegan things I still have to tr...,3
4,U0,This vegan fried chicken from KFC is on the sp...,"Omg you HAVE to try, the vegan fried chicken f...",4
...,...,...,...,...
95,U9,Just having this vegan hotdog from the school ...,The vegan hotdog from the school canteen is so...,5
96,U9,"Just had my first vegan cake at ""groene bakker...",I’ve just tries this vegan cake from the “groe...,6
97,U9,This vegan chocolate is on point. Its with oat...,This oatmilk chocolate is so good! With some c...,7
98,U9,I'm all about fast food. I'm having some fries...,I really like fast food. I’m eating a veggie b...,8


### Step 2 LLM TST

In [62]:
# This calls Mistral API
df_llm_tst = p.llm_tst(df_user_data,neutral_sentences[0:2])
df_llm_tst.head(3)

,username,id_neutral_sentence,neutral_sentence,tst_id,tst_sentence,explanation,llm_tst,query,model,prompt_tokens,completion_tokens,object,promptID,timestamp,output_run
0,U0,0,I’m all about that food. I usually kick off th...,U020240621162514,"I'm totally into food, you know? I always star...","Person X uses colloquial language such as ""I'...","tst\_sentence: ""I'm totally into food, you kno...",\n[INST]You are an expert in text style transf...,mistral-small,325,111,chat.completion,2,20240621162514,run_20240621162511
1,U0,1,Just getting my vitamins in at the school cant...,U020240621162516,So I'm just grabbing my vitamins at the school...,"In this conversational style of person X, cas...","tst\_sentence: ""So I'm just grabbing my vitami...",\n[INST]You are an expert in text style transf...,mistral-small,329,110,chat.completion,2,20240621162516,run_20240621162511
2,U1,0,I’m all about that food. I usually kick off th...,U120240621162518,"No worries about food, I've got it covered! Ju...","In the style of person X, the sentence is mor...","tst\_sentence: ""No worries about food, I've go...",\n[INST]You are an expert in text style transf...,mistral-small,322,102,chat.completion,2,20240621162518,run_20240621162511


### Step 3 LLM Evaluate

In [28]:
# In case you wanna read other csv files
# df_llm_tst = pd.read_csv('f6_llm_tst_data/run_20240604131627/run_20240604131627_tst_postprocess.csv')

In [63]:
df_eval,df_eval_raw = p.llm_evl(df_llm_tst,user_sentences)
df_eval.head(3)

Evaluating TST sentences: 100%|██████████| 20/20 [03:45<00:00, 11.28s/it]


Exception at index: 43  
 with value: Score: 2_somewhat concise/descriptive

Explanation: The text contains some descriptive elements, such as "good plant-based meal," "tasty beans," and "vegan sausage," which provide some vividness to the image. However, the text is predominantly conversational and lacks the level of detail typically found in descriptive text, hence the rating of "somewhat concise/descriptive."
Exception at index: 113  
 with value: Score: 2_somewhat concise/descriptive

Explanation: The text is somewhat concise because it gets straight to the point about the person's preference for plant-based meals and their routine of starting the week with a particular meal. However, it also contains some descriptive elements, such as "good plant-based meal" and "delicious combo of beans and vegan sausage," which help to create a slight image or evoke a mild emotion around the subject. The text is not overly descriptive, but it goes beyond just stating the facts and adds a bit of 

,tst_id,tst_sentence,username,id_neutral_sentence,user_sentence,eval_score_formality,timestamp_score_formality,eval_explanation_formality,eval_score_descriptiveness,timestamp_score_descriptiveness,...,eval_score_comprehensibility,timestamp_score_comprehensibility,eval_explanation_fluency_comprehensibility,eval_score_topic_similarity,timestamp_score_topic_similarity,eval_explanation_topic_similarity,eval_score_meaning_similarity,timestamp_score_meaning_similarity,eval_explanation_meaning_similarity,output_run
0,U020240621162514,"I'm totally into food, you know? I always star...",U0,0,I love food! I really like to have beans with ...,1,20240621162602,This text is predominantly informal. The use o...,2,20240621162604,...,3,20240621162609,"The text is written entirely in English, so it...",3,20240621162611,"Both texts have the same topic, which is the c...",3,20240621162613,The two texts have the same meaning. Both expr...,run_20240621162511
1,U020240621162516,So I'm just grabbing my vitamins at the school...,U0,1,"I am going for a something healthy today, quin...",1,20240621162615,This text contains several features that indic...,1,20240621162616,...,3,20240621162621,"The text is written entirely in English, so it...",3,20240621162622,"The two texts are about the same topic, which ...",3,20240621162623,"Both texts convey the same information, statin...",run_20240621162511
2,U120240621162518,"No worries about food, I've got it covered! Ju...",U1,0,I really love good food! I tend to start my we...,2,20240621162625,The text contains some informal elements such ...,2,20240621162627,...,3,20240621162632,"The text is written in English, so it gets a s...",2,20240621162633,Both texts mention starting the day with beans...,2,20240621162635,The two texts have somewhat similar meaning. B...,run_20240621162511
